# SentiScope

## Data Import and Pre-processing

In [ ]:
# Imports
import os
import bz2
import pandas as pd
import string
import re

In [2]:
path = "../raw_data/"
file_path = os.path.join(path, "train.ft.txt.bz2")
data = []
with bz2.open(file_path, "rt", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(" ", 1)
        if len(parts) == 2:
            label, text = parts
            label = label.replace("__label__", "")
            data.append((label, text))
df = pd.DataFrame(data, columns=["label", "text"])
df["label"] = df["label"].astype(int)
df.head()

,label,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."


In [10]:
display(df.shape, df["label"].value_counts(normalize=True))

(3600000, 2)

label
2    0.5
1    0.5
Name: proportion, dtype: float64

In [ ]:
test_df = df.sample(frac=0.01)
display(test_df.head(), test_df.shape)

,label,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."


(1000, 2)

In [ ]:
#Basic cleaning function
def basic_cleaning(text):


In [23]:
test_df["clean_text"] = df["text"].apply(basic_cleaning)
display(test_df.head())

KeyboardInterrupt: 